In [16]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [17]:
df = pd.read_csv("/content/Dataset_timeseries.csv")
data = df[['Time (s)', 'SOPAS', 'Number']]

# Standardize the features
scaler = StandardScaler()
data[['Time (s)', 'SOPAS']] = scaler.fit_transform(data[['Time (s)', 'SOPAS']])

In [18]:
def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(len(data) - seq_length):
        X.append(data.iloc[i:i+seq_length, :-1].values)  # Exclude the target column
        y.append(data.iloc[i+seq_length, -1])  # Target is the 'Number' column
    return np.array(X), np.array(y)

# Set sequence length
SEQ_LENGTH = 5

# Create sequences
X, y = create_sequences(data, SEQ_LENGTH)

In [19]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# One-hot encode the labels
y_train = to_categorical(y_train, num_classes=4)
y_test = to_categorical(y_test, num_classes=4)

In [20]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(SEQ_LENGTH, 2)))
model.add(LSTM(50))
model.add(Dense(4, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 5, 50)             10600     
                                                                 
 lstm_7 (LSTM)               (None, 50)                20200     
                                                                 
 dense_3 (Dense)             (None, 4)                 204       
                                                                 
Total params: 31004 (121.11 KB)
Trainable params: 31004 (121.11 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20
4944/5117 [===========================>..] - ETA: 0s - loss: 0.5533 - accuracy: 0.7588

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')